In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


/Users/aayeshaaayesha/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
import pandas as pd
df_profile1_sample = pd.read_excel('friend_profile1_gender.xlsx')
df_profile2_sample = pd.read_excel('friend_profile2_gender.xlsx')




In [4]:
from statistics import mean 
col_list = ['accomp1f']
def sentiForCol(col_list):
    pos_mean_ls = []
    for k in range(df_profile1_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_profile1_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol(col_list))
profile1 = sentiForCol(col_list)

[0.1863101, 0.022276387, 0.18364666, 0.2088199]


In [5]:
col_list = ['accomp1f']
def sentiForCol1(col_list):
    pos_mean_ls = []
    for k in range(df_profile2_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_profile2_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol1(col_list))
profile2 = sentiForCol1(col_list)

[0.080056295, 0.60055566, 0.9117711, 0.316021]


In [6]:
df_profile1_interface_t_test = pd.DataFrame({'profile': 'profile1', 'interface_pos': profile1 })
df_profile2_interface_t_test = pd.DataFrame({'profile': 'profile2', 'interface_pos': profile2 })


In [7]:
df_profiles_combine_interface_t_test = pd.concat([df_profile1_interface_t_test, df_profile2_interface_t_test], axis=0)
df_profiles_combine_interface_t_test

,profile,interface_pos
0,profile1,0.186310
1,profile1,0.022276
2,profile1,0.183647
3,profile1,0.208820
0,profile2,0.080056
1,profile2,0.600556
2,profile2,0.911771
3,profile2,0.316021


In [8]:
from scipy import stats
from itertools import combinations
hobbies = df_profiles_combine_interface_t_test['profile'].unique()

# Perform pairwise t-tests
pairwise_results = []
for (hobby1, hobby2) in combinations(hobbies, 2):
    group1 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == hobby1]['interface_pos']
    group2 = df_profiles_combine_interface_t_test[df_profiles_combine_interface_t_test['profile'] == hobby2]['interface_pos']
    t_stat, p_value = stats.ttest_ind(group1, group2)
    pairwise_results.append(((hobby1, hobby2), t_stat, p_value))

# Print results
for (hobby1, hobby2), t_stat, p_value in pairwise_results:
    print(f"{hobby1} vs. {hobby2}: T-statistic = {t_stat:.2f}, P-value = {p_value:.4f}")


profile1 vs. profile2: T-statistic = -1.77, P-value = 0.1274
